In [207]:
import pandas as pd
import numpy as np
import ast
import seaborn as sns
import matplotlib.pyplot as plt
import collections
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter, defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer

In [196]:
# dataframe
df = pd.read_csv('superheroes_nlp_dataset.csv')
df.head()

,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,durability_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,3-D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...",NaN,85,30,60,60,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,514A (Gotham),Bruce Wayne,NaN,10,He was one of the many prisoners of Indian Hil...,NaN,100,20,30,50,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,A-Bomb,Richard Milhouse Jones,Richard Milhouse Jones,20,"Richard ""Rick"" Jones was orphaned at a young ...","On rare occasions, and through unusual circu...",80,100,80,100,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
3,Aa,Aa,NaN,12,Aa is one of the more passive members of the P...,NaN,80,50,55,45,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aaron Cash,Aaron Cash,Aaron Cash,5,Aaron Cash is the head of security at Arkham A...,NaN,80,10,25,40,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


1)	How would you define the most powerful superhero from the information available in dataset?
a.	Who is the most powerful superhero of each creator?
The superhero with a higher overal score is considered more powerful.

In [201]:
def clean_score(value):
    try:
        return int(value)
    except (ValueError, TypeError):
        return np.nan

df['overall_score'] = df['overall_score'].apply(clean_score)

# remove rows with non-integer values
df_cleaned = df.dropna(subset=['overall_score'])
most_powerful_superheros = df_cleaned.groupby('creator')['name','overall_score'].max()

print(f"The most powerful superheroe of each creator is: {most_powerful_superheros}")


The most powerful superheroe of each creator is:                                               name  overall_score
creator                                                          
ABC Studios                       Stephanie Powell            7.0
Blizzard Entertainment                       A'dal            7.0
Capcom                                        Zero           32.0
Cartoon Network                       Rick Sanchez           19.0
Clive Barker                               Pinhead           21.0
DC Comics                            Zoom (New 52)          237.0
Dark Horse Comics                      Violet Parr           38.0
Disney                                  Stan Pines           13.0
Dreamworks                              Jack Frost            9.0
George Lucas                                  Yoda           84.0
George R. R. Martin             The Smiling Knight           13.0
Hanna-Barbera                              Birdman            9.0
Hasbro                     

<ipython-input-201-e0da9547d27c>:11: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  most_powerful_superheros = df_cleaned.groupby('creator')['name','overall_score'].max()


2)	Find the top 5 superpowers in descending order.

In [202]:
superpower_dict = defaultdict(int)

for idx, row in df.iterrows():
    for item in ast.literal_eval(df.iloc[idx]['superpowers']):
        superpower_dict[item] += 1
        
# sort occurances of superpowers in descending order
sorted_superpower_dict = dict(sorted(superpower_dict.items(), key=lambda item: item[1], reverse = True))

# top 5 superpowers in descending order
top_5_superpowers = []
count = 0

for key, val in sorted_superpower_dict.items():
    if count < 5:
        top_5_superpowers.append(key)
        count += 1
    else:
        break
print(f"top 5 superpowers are: {top_5_superpowers}")

top 5 superpowers are: ['Agility', 'Stamina', 'Super Strength', 'Durability', 'Intelligence']


3)	Which race has the most immortal superheroes?

In [203]:
# group by race and count immortality for each race
race_immortality_count = df.groupby('type_race')['has_immortality'].sum()
most_immortal_race = race_immortality_count.idxmax()
immortal_superheroes_max_count = race_immortality_count.max()
print(f"The race with the most immortal superheroes is {most_immortal_race} with {immortal_superheroes_max_count}")

The race with the most immortal superheroes is God / Eternal with 36.0


4)	Name the creator having most superheroes of type “Parademon”.

In [204]:
parademon_df = df[df['type_race'] == 'Parademon']

# group by creator and count the number of Parademon superheroes for each creator
creator_parademon_count = parademon_df.groupby('creator')['type_race'].count()

# finding the creator with the most Parademon superheroes
most_parademon_creator = creator_parademon_count.idxmax()
num_parademon_superheroes = creator_parademon_count.max()

print(f"The creator with the most Parademon superheroes is {most_parademon_creator} with {num_parademon_superheroes} Parademon superheroes.")

The creator with the most Parademon superheroes is DC Comics with 1 Parademon superheroes.


5)	Which comic creator has the most superhero teams?
a.	Find names, real names and alias of superhero who is part of most teams.
b.	Are there any crossovers between creators and teams?


In [124]:
df_expanded = df.explode('teams')
creator_team_counts = df_expanded.groupby('creator')['teams'].nunique()

# creator with the most teams
most_teams_creator = creator_team_counts.idxmax()
num_teams = creator_team_counts.max()

print(f"The creator with the most superhero teams is {most_teams_creator} with {num_teams} teams.")

# a) names, real names and alias of superhero who is part of most teams
most_teams_superhero = df_expanded['teams'].value_counts().idxmax()
superhero_data = df_expanded[df_expanded['teams'] == most_teams_superhero]

superhero_name = superhero_data['name'].values[0]
real_name = superhero_data['real_name'].values[0]
aliases = superhero_data['aliases'].values[0]

print(f"The superhero with the most teams is {superhero_name}.")
print(f"Real Name: {real_name}")
print(f"Aliases: {aliases}")

# b) crossover
crossovers = df_expanded.groupby('teams')['creator'].nunique().gt(1)

for team, is_crossover in crossovers[crossovers].items():
    if ast.literal_eval(team):
        team_creators = df_expanded[df_expanded['teams'] == team]['creator'].astype(str).tolist()
        print(f"Crossovers: ")
        print(f"Team: {team}, Creators: {', '.join(team_creators)}")

The creator with the most superhero teams is Marvel Comics with 241 teams.
The superhero with the most teams is 514A (Gotham).
Real Name: Bruce Wayne
Aliases: ['Subject 514A', 'Bruce Wayne', 'Bruce 2']
Crossovers: 
Team: ['Incredible Family'], Creators: Dark Horse Comics, Dark Horse Comics, Dark Horse Comics, Disney, Dark Horse Comics, Dark Horse Comics


6)	What are the characteristics that can predict a superhero alignment?

We can train a logistic regression model, with alignment as target value and try training models with different feaqtures to find the most contributing feature to the model performance.

7)	From history of superheroes, 
a.	Find list of superheroes having negative past but now aligned positively.
(Negativity of past can be decided by multiple methods, please use which is familiar to you)
b.	Extract patterns from superhero history for each creator.


In [160]:
# negative ontologies that can be defined using domain expert
negative_keywords = ["Villain", "Criminal", "Evil", "Dark Past", "Sinister", "Reckless", "Corrupt", "Antihero",
                    "Betrayal", "Conflict", "Tragic", "Moral Ambiguity", "Redemption", "Complicated Past",
                    "Fallen", "Controversial", "Suffering", "Conflict with Authorities", "Tragic Origin", "Regret", "Agressive"]

negative_pattern = '|'.join(negative_keywords)
df['history_text'].fillna('', inplace=True)


# superheroes with negative past
superheroes_neg_past = df[(df['history_text'].str.contains(negative_pattern, case=False, regex=True))]

# negative past, positive alignment
superheroes_neg_past_pos_alignment =  superheroes_neg_past[(superheroes_neg_past['alignment'] == 'Good')]
superheroes_neg_past_pos_alignment = superheroes_neg_past_pos_alignment['name'].tolist()

print("Superheroes with a Negative Past but Positive Alignment:")
print(len(superheroes_neg_past_pos_alignment))

Superheroes with a Negative Past but Positive Alignment:
345


b. Extract patterns from superhero history for each creator.

In [209]:
# group by creator
creator_groups = df.groupby('creator')

# stopwords
stop_words = set(stopwords.words('english'))

# text preprocessing
def preprocess_text(text):
    if pd.isna(text):
        return []  
    tokens = word_tokenize(text.lower())
    filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    return filtered_tokens

for creator, group in creator_groups:
    histories = group['history_text'].tolist()
    
    # histories
    preprocessed_histories = [preprocess_text(history) for history in histories]
    
    # word frequency
    word_frequency = Counter(word for history_tokens in preprocessed_histories for word in history_tokens)
    
    # top 10 most common words for each creator
    print(f"Creator: {creator}")
    print("Top 10 Most Common Words are:")
    for word, count in word_frequency.most_common(10):
        print(f"{word}: {count}")


Creator: ABC Studios
Top 10 Most Common Words are:
family: 8
jim: 7
powers: 6
stephanie: 6
world: 5
daphne: 4
finds: 4
new: 4
discovers: 3
ca: 3
Creator: Blizzard Entertainment
Top 10 Most Common Words are:
shattrath: 4
khadgar: 2
point: 2
city: 2
forces: 2
naaru: 1
little: 1
known: 1
personally: 1
though: 1
Creator: Capcom
Top 10 Most Common Words are:
zero: 312
x: 141
wesker: 85
sigma: 63
would: 51
maverick: 50
however: 47
virus: 45
umbrella: 42
elf: 38
Creator: Cartoon Network
Top 10 Most Common Words are:
morty: 2
rick: 2
sanchez: 2
grandson: 1
assistant: 1
adventure: 1
space: 1
main: 1
character: 1
leading: 1
Creator: Clive Barker
Top 10 Most Common Words are:
batwing: 2
emery: 1
schaub: 1
employed: 1
fry: 1
cook: 1
morganton: 1
north: 1
carolina: 1
unknown: 1
Creator: DC Comics
Top 10 Most Common Words are:
batman: 862
time: 734
one: 729
superman: 672
would: 662
new: 656
robin: 553
back: 527
city: 512
later: 510
Creator: Dark Horse Comics
Top 10 Most Common Words are:
hellboy: 85

8)	Report on the 10 superheroes with most relatives, status of those relatives where possible, and the alignment of those superheroes.

In [195]:
def parse_relatives_and_statuses(relatives):
    if pd.isna(relatives):
        return []  
    
    relatives_list = relatives.split('; ')
    relatives_info = []
    
    for relative_info in relatives_list:
        relative_parts = relative_info.split(' (')
        if len(relative_parts) > 1:
            relative_name = relative_parts[0]
            relative_status = relative_parts[1][:-1]  
        else:
            relative_name = relative_parts[0]
            relative_status = 'unknown'  
        
        relatives_info.append((relative_name, relative_status))
    
    return relatives_info

# extracting relatives and statuses
df['relatives_stat'] = df['relatives'].apply(parse_relatives_and_statuses)

# number of relatives for each superhero
df['relative_count'] = df['relatives_stat'].apply(len)

# top 10 superheroes
top_10_superheroes = df.nlargest(10, 'relative_count')

for index, row in top_10_superheroes.iterrows():
    print(f"Superhero: {row['name']}")
    
    alignment = row['alignment']
    print(f"The alignment is: {alignment}")
    
    relatives_info = row['relatives_stat']
    for relative_name, relative_status in relatives_info:
        print(f"Relative: {relative_name}, Status: {relative_status}")
    


Superhero: Cyclops
The alignment is: Good
Relative: Oscar Summers, Status: adoptive paternal distant ancestor, deceased
Relative: Daniel Summers, Status: paternal distant ancestor, deceased
Relative: Amanda Mueller, Status: Black Womb
Relative: unnamed great-grandfather, deceased, Status: unknown
Relative: Gloria Dayne, Status: Fontanelle
Relative: Philip Summers, Status: paternal grandfather
Relative: Deborah Summers, Status: paternal grandmother
Relative: Christopher Summers, Status: Corsair
Relative: Katherine Ann Summers, Status: mother, deceased
Relative: Jack Winters, Status: Jack O'Diamonds
Relative: Alexander Summers, Status: Havok
Relative: Gabriel Summers, Status: Vulcan
Relative: Madelyne Pryor-Summers, Status: Red Queen
Relative: Jean Grey-Summers, Status: Phoenix
Relative: Nathan Summers, Status: Cable
Relative: Gailyn Bailey, Status: niece, deceased
Relative: Joseph Bailey, Status: nephew, deceased
Relative: Adam Neramani, Status: X-Treme
Relative: Hope Summers, Status: a